In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pdb; 

In [ ]:
### install detectron2 and dependency 

!pip install pyyaml==5.1
# This is the current pytorch version on Colab. Uncomment this if Colab changes its pytorch version
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9")   # please manually install torch 1.9 if Colab changes its default version

In [ ]:
# install geospatial dependencies

!pip -q install rasterio
!pip -q install fiona
!pip -q install geopandas

In [ ]:
# import libraries for visualisation

!pip -q install holoviews geoviews hvplot

In [ ]:
# Import necessary parts of detectron2
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode
from detectron2.engine import DefaultTrainer

# necessary libraries
import pandas as pd
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
import json
import glob
import urllib

# import geospatial libraries
import rasterio
from rasterio.transform import from_origin
import rasterio.features

import fiona

from shapely.geometry import shape, mapping, box
from shapely.geometry.multipolygon import MultiPolygon

#from google.colab.patches import cv2_imshow

# import visualisation libraries

import holoviews as hv
from IPython.display import display
#import geoviews.tile_sources as gts

#import hvplot.pandas
#import hvplot.xarray

#hv.extension('bokeh', width=100)

In [ ]:
# Make sure the inevitable error messages are useful.
CUDA_LAUNCH_BLOCKING="1"

In [ ]:
# define the URL to retrieve the model from Zenodo
#fn = 'model_final.pth'
#url = f'https://zenodo.org/record/5515408/files/{fn}?download=1'

#urllib.request.urlretrieve(url, '/content/drive/Shareddrives/detectreegb/weights' + '/' + fn)

In [ ]:
# Setup to predict on new images, here setting up for the trees_test dataset, but can also use this setup
# for predicting on individual images as seen 2 cells down

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

### path to the saved pre-trained model weights
cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/Model/model_final.pth'

# set confidence threshold at which we predict
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.15

predictor = DefaultPredictor(cfg)

In [ ]:
# make and visualise predictions, on a particular file, just to check these weights are working ok 

# give the image that we are trying to predict on

im = cv2.imread("/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/FullTile/tiles/pngs/TrainingTiles/tile_287176_584157.png")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1], scale=0.3, instance_mode=ColorMode.IMAGE_BW)   # remove the colors of unsegmented pixels
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
image = cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
display(Image.fromarray(image))


In [ ]:

# save this as jpg or png...we are going for png...again, named with the origin of the specific tile
#cv2.imwrite('/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/tiles/pngs/tile_try111.png', image[1])
image[2]

In [ ]:
# predict on all pngs and convert predictions to shapefiles

import glob

for filepath in glob.iglob('/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/FullTile/tiles/pngs/TrainingTiles/*.png'):
    print(filepath)
    img = cv2.imread(filepath)
    outputs = predictor(img)
    mask_array = outputs['instances'].pred_masks.cpu().numpy()

    # get confidence scores too 
    mask_array_scores = outputs['instances'].scores.cpu().numpy()

    num_instances = mask_array.shape[0]
    mask_array_instance = []
    output = np.zeros_like(mask_array) 

    mask_array_instance.append(mask_array)
    output = np.where(mask_array_instance[0] == True, 255, output)
    fresh_output = output.astype(np.uint8)
    x_scaling = 140/fresh_output.shape[1]
    y_scaling = 140/fresh_output.shape[2]
    
    # this is an affine transform. This needs to be altered significantly.
    transform = from_origin(int(filepath[-17:-11])-20, int(filepath[-10:-4])+120, y_scaling, x_scaling)

    
    output_raster =  '/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/rasters/Training50/' + 'predicted_raster_' + filepath[-17:-4]+ '.tif'

    new_dataset = rasterio.open(output_raster, 'w', driver='GTiff',
                                    height = fresh_output.shape[1], width = fresh_output.shape[2], count = fresh_output.shape[0],
                                    dtype=str('uint8'),
                                    crs='+proj=utm +zone=50 +datum=WGS84 +units=m +no_defs',  
                                    transform=transform)
   
    new_dataset.write(fresh_output)
    new_dataset.close()

    with rasterio.open(output_raster) as src:
        shp_schema = {'geometry': 'MultiPolygon','properties': {'pixelvalue': 'int', 'score': 'float'}}    

        crs = src.crs
        for i in range(src.count):
            src_band = src.read(i+1)
            src_band = np.float32(src_band)
            conf = mask_array_scores[i-2]
            # Keep track of unique pixel values in the input band
            unique_values = np.unique(src_band)
            # Polygonize with Rasterio. `shapes()` returns an iterable
            # of (geom, value) as tuples
            shapes = list(rasterio.features.shapes(src_band, transform=src.transform))
        
     


            if i == 0:
                with fiona.open('/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/shapes/' + 'predicted_polygons_' + filepath[-17:-4] + '_' + str(0) + '.shp', 'w', 'ESRI Shapefile', shp_schema, crs) as shp:
                    polygons = [shape(geom) for geom, value in shapes if value == 255.0]                                        
                    multipolygon = MultiPolygon(polygons)
                            # simplify not needed here
                            #multipolygon = multipolygon_a.simplify(0.1, preserve_topology=False)                    
                    shp.write({
                              'geometry': mapping(multipolygon),
                              'properties': {'pixelvalue': int(unique_values[1]), 'score': float(conf)} 
                              })
            else:
                with fiona.open('/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/shapes/' + 'predicted_polygons_' + filepath[-17:-4] + '_' + str(0)+'.shp', 'a', 'ESRI Shapefile', shp_schema, crs) as shp:
                    polygons = [shape(geom) for geom, value in shapes if value == 255.0]                                        
                    multipolygon = MultiPolygon(polygons)
                            # simplify not needed here
                            #multipolygon = multipolygon_a.simplify(0.1, preserve_topology=False)                    
                    shp.write({
                              'geometry': mapping(multipolygon),
                              'properties': {'pixelvalue': int(unique_values[1]), 'score': float(conf)} 
                              })
        


/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/FullTile/tiles/pngs/TrainingTiles/tile_286400_583762.png
/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/FullTile/tiles/pngs/TrainingTiles/tile_286400_583662.png
/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/FullTile/tiles/pngs/TrainingTiles/tile_286559_583854.png
/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/FullTile/tiles/pngs/TrainingTiles/tile_286748_583830.png
/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/FullTile/tiles/pngs/TrainingTiles/tile_286716_583929.png
/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/FullTile/tiles/pngs/TrainingTiles/tile_286818_583948.png
/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/FullTile/tiles/pngs/TrainingTiles/tile_287000_583744.png
/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/FullTile/tiles/pngs/TrainingTiles/tile_287153_583851.png
/content/drive/MyDrive/f

In [ ]:
### we can group all our shapefiles as shown in this cell

import pandas as pd
import geopandas as gpd

gdf1 = gpd.read_file('/content/drive/MyDrive/Output/sepilok/shapes/predicted_polygons_602800_647000_0.shp')
gdf2 = gpd.read_file('/content/drive/MyDrive/Output/sepilok/shapes/predicted_polygons_602800_647200_0.shp')
gdf3 = gpd.read_file('/content/drive/MyDrive/Output/sepilok/shapes/predicted_polygons_602800_647300_0.shp')

grouped_gdf = gpd.GeoDataFrame(pd.concat([gdf1, gdf2, gdf3]))

In [ ]:
grouped_gdf

,pixelvalue,score,geometry
0,255,0.157371,"POLYGON ((602874.100 647093.900, 602876.500 64..."
1,255,0.154994,"POLYGON ((602816.300 647023.800, 602819.600 64..."
2,255,0.977144,"POLYGON ((602862.400 647026.200, 602863.900 64..."
3,255,0.970589,"POLYGON ((602850.600 647069.400, 602853.500 64..."
4,255,0.946630,"POLYGON ((602846.900 647048.500, 602847.000 64..."
...,...,...,...
30,255,0.208750,"POLYGON ((602900.800 647369.600, 602902.600 64..."
31,255,0.194811,"POLYGON ((602800.200 647356.200, 602800.800 64..."
32,255,0.190992,"POLYGON ((602810.100 647327.400, 602811.000 64..."
33,255,0.163410,"POLYGON ((602889.700 647410.700, 602892.600 64..."


In [ ]:
# then we can create interactive plots as shown here

plot_vector = grouped_gdf.hvplot(hover_cols=['score']).opts(fill_color=None,line_color='red',alpha=0.8, width=800, height=600)

plot_vector

/usr/local/lib/python3.7/dist-packages/geoviews/util.py:379: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for g in geom:
/usr/local/lib/python3.7/dist-packages/geoviews/util.py:332: ShapelyDeprecationWarning: The 'array_interface_base' property is deprecated and will be removed in Shapely 2.0.
  arr = geom.array_interface_base['data']


TypeError: ignored

TypeError: ignored

In [ ]:
# We can plot tiles of the tiff too

# load and plot RGB image
r = tc_rgb.sel(band=[1,2,3])

normalized = r/(r.quantile(.99,skipna=True)/255)

mask = normalized.where(normalized < 255)

int_arr = mask.astype(int)

plot_rgb = int_arr.astype('uint8').hvplot.rgb(
    x='x', y='y', bands='band', data_aspect=1
)

plot_rgb

NameError: ignored

In [ ]:
# we can save the plot too

hvplot.save(plot_vector, '/content/drive/Shareddrives/detectreegb/sepilok/outputs/interactive_plots/combined_plot.html')

In [ ]:
# load and plot polygons
in_shp = glob.glob(config['out_shapefile'] + '/*.shp')

poly_df = gpd.read_file(in_shp[0])

plot_vector = poly_df.hvplot(hover_cols=['score']).opts(fill_color=None,line_color='red',alpha=0.8, width=800, height=600)

plot_vector

NameError: ignored